## Multi-dimensional image data

In [1]:
import fastplotlib as fpl
from sidecar import Sidecar
from ipywidgets import VBox, HBox, IntSlider

# Sliders to scroll through image data

We often already have large image arrays (whether in RAM or through lazy loading), and want to view 2D frames across one or more dimensions. There is an `ImageWidget` that should really be used for this, but this example just shows how you can use `ipywidgets` to change data or any **`GraphicFeature`**

### Some code to generate a bunch of time-varying Gaussians. This code is NOT important for understanding `fastplotlib`, it just generates some video-like data for us to visualize!

In [2]:
import numpy as np
from scipy.stats import multivariate_normal

# set up gaussians centered at component_centers
n_frames = 1000
spatial_dims = 512

frame_shape = [512, 512]

n_components = 32
component_centers = (np.random.rand(n_components, 2) * spatial_dims).astype(int)

# create component images: stack of images one for ech component
spatial_sigma = 50
x, y = np.meshgrid(
    np.arange(0, spatial_dims),
    np.arange(0, spatial_dims)
)

pos = np.dstack((x, y))
component_sigma = np.array(
    [[spatial_sigma, 0],
     [0, spatial_sigma]]
)

component_images = []
for comp_ix in range(n_components):
    comp_mean = component_centers[comp_ix]
    gauss_rep = multivariate_normal(comp_mean, component_sigma)
    gauss_img = gauss_rep.pdf(pos)
    component_images.append(gauss_img)

component_images = np.array(component_images)


# generate traces
tau = 10
max_amp = 2000
amps_all = []

for component_num in range(n_components):
    amps = []
    amp = 0
    for time_step in np.arange(n_frames):
        if np.random.uniform(0,1) > 0.98:
            amp = max_amp
        else:
            amp = np.max(np.array([amp - amp/tau, 0]));
        amps.append(amp)
    amps = np.array(amps)
    amps_all.append(amps)
amps_all = np.array(amps_all)

# create movie
movie = np.zeros((n_frames, spatial_dims, spatial_dims))
for frame_ix in np.arange(n_frames):
    for comp_ix in range(n_components):
        movie[frame_ix] += amps_all[comp_ix][frame_ix] * component_images[comp_ix]

### Now we have a movie of the following shape, an image sequence

In [3]:
movie.shape

(1000, 512, 512)

### This is usually [time, x, y]

## Plot and scroll through the first dimension with a slider

In [4]:
plot_movie = fpl.Plot()

# plot the first frame to initialize
movie_graphic = plot_movie.add_image(movie[0], vmin=0, vmax=movie.max(), cmap="gnuplot2")

# make a slider
slider = IntSlider(min=0, max=movie.shape[0] - 1, step=1, value=0)

# function to update movie_graphic
def update_movie(change):    
    index = change["new"]
    movie_graphic.data = movie[index]
    
slider.observe(update_movie, "value")
    
# Use an ipywidgets VBox to show the plot and slider
VBox([plot_movie.show(), slider])

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


#### Note that the use of globals in the `update_movie()` here can get messy, this is not recommended and  you should create a class to properly handle combining widgets like this. _However_ if you want slider widgets for imaging data the recommended way to do this is by using the `ImageWidget`.

## Single image sequence `ImageWidget`

In [12]:
a = np.random.rand(500, 512, 512)

In [13]:
iw = fpl.ImageWidget(
    data=a,
    vmin_vmax_sliders=True,
    cmap="viridis"
)

RFBOutputContext()

In [14]:
sc = Sidecar()

with sc:
    display(iw.show())

### can dynamically change features

In [15]:
iw.gridplot[0, 0].graphics[0].cmap = "gnuplot2"

### Play with setting different window functions

These can also be given as kwargs to `ImageWidget` during instantiation

In [16]:
# must be in the form of {dim: (func, window_size)}
iw.window_funcs = {"t": (np.mean, 13)}

In [17]:
# change the winow size
iw.window_funcs["t"].window_size = 23

In [18]:
# change the function
iw.window_funcs["t"].func = np.max

In [19]:
# or set it again
iw.window_funcs = {"t": (np.min, 11)}

### Can also set new data

In [20]:
new_data = np.random.rand(500, 512, 512)
iw.set_data(new_data=new_data)

In [23]:
sc.close()

# Gridplot of txy data

In [24]:
dims = (100, 512, 512)
data = [np.random.rand(*dims) for i in range(4)]

In [25]:
iw = fpl.ImageWidget(
    data=data, 
    slider_dims=["t"], 
    # dims_order="txy", # you can set this manually if dim order is not the usual
    vmin_vmax_sliders=True,
    names=["zero", "one", "two", "three"],
    window_funcs={"t": (np.mean, 5)},
    cmap="gnuplot2", 
)

RFBOutputContext()

### pan-zoom controllers are all synced in a `ImageWidget`

In [27]:
sc = Sidecar()

with sc:
    display(iw.show())

### Index the subplots using the names given to `ImageWidget`

In [28]:
iw.gridplot["two"]

two: Subplot @ 0x7ff628670e50
  parent: None
  Graphics:
	'image_widget_managed': ImageGraphic @ 0x7ff6282bd550

### change window functions just like before

In [29]:
iw.window_funcs["t"].func = np.max

### change features just like before

In [31]:
iw.gridplot["zero"].graphics[0].cmap = "viridis"